In [1]:
import tensorflow as tf

from tensorflow.keras import layers

from sklearn.metrics import accuracy_score

In [2]:
TRAIN_VAL_DIR = "/kaggle/input/intel-image-classification/seg_train/seg_train/"
TEST_DIR = "/kaggle/input/intel-image-classification/seg_test/seg_test/"
IMAGE_SHAPE = (128, 128, 3)
NUM_CLASSES = 6
EPOCHS = 150
BATCH_SIZE = 32

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_VAL_DIR,
    batch_size=None,
    image_size=IMAGE_SHAPE[:-1],
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_VAL_DIR,
    batch_size=None,
    image_size=IMAGE_SHAPE[:-1],
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    batch_size=None,
    image_size=IMAGE_SHAPE[:-1],
    shuffle=False,
)

Found 14034 files belonging to 6 classes.
Using 11228 files for training.
Found 14034 files belonging to 6 classes.
Using 2806 files for validation.
Found 3000 files belonging to 6 classes.


In [4]:
train_ds = train_ds.map(lambda x, y: (tf.cast(x, dtype=tf.float32) / 255., y))
val_ds = val_ds.map(lambda x, y: (tf.cast(x, dtype=tf.float32) / 255., y))
test_ds = test_ds.map(lambda x, y: (tf.cast(x, dtype=tf.float32) / 255., y))

In [5]:
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [6]:
def build_cnn(num_classes):
    cnn_layers = [
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=IMAGE_SHAPE),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes)
    ]

    model = tf.keras.Sequential(cnn_layers)
    
    return model

In [7]:
model = build_cnn(NUM_CLASSES)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 124, 124, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 32)       0         
 )                                                               
                                                        

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [9]:
model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=EPOCHS,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

Epoch 1/150


2023-04-29 02:50:41.010764: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


351/351 [==============================] - 55s 113ms/step - loss: 1.3714 - accuracy: 0.5615 - val_loss: 3.7617 - val_accuracy: 0.3140
Epoch 2/150
351/351 [==============================] - 17s 45ms/step - loss: 0.8976 - accuracy: 0.6808 - val_loss: 2.3356 - val_accuracy: 0.4783
Epoch 3/150
351/351 [==============================] - 16s 45ms/step - loss: 0.7491 - accuracy: 0.7382 - val_loss: 2.5388 - val_accuracy: 0.4861
Epoch 4/150
351/351 [==============================] - 17s 45ms/step - loss: 0.6905 - accuracy: 0.7562 - val_loss: 3.2116 - val_accuracy: 0.4658
Epoch 5/150
351/351 [==============================] - 16s 45ms/step - loss: 0.6128 - accuracy: 0.7804 - val_loss: 2.7592 - val_accuracy: 0.5513
Epoch 6/150
351/351 [==============================] - 17s 46ms/step - loss: 0.5528 - accuracy: 0.8008 - val_loss: 1.9945 - val_accuracy: 0.5862
Epoch 7/150
351/351 [==============================] - 17s 46ms/step - loss: 0.5360 - accuracy: 0.8097 - val_loss: 2.2579 - val_accuracy: 0.6

In [10]:
predictions = model.predict(test_ds.map(lambda x, y: x))
predictions = tf.argmax(tf.math.softmax(predictions), axis=1).numpy()
predictions = predictions.tolist()

94/94 [==============================] - 8s 82ms/step


In [11]:
y_test = list(test_ds.unbatch().map(lambda x, y: y).as_numpy_iterator())

In [14]:
print(f"{round(accuracy_score(y_test, predictions) * 100, 3)}% accuracy on test set.")

82.9% accuracy on test set.
